In [ ]:
import pandas as pd

In [ ]:
skiAreaReviewDF = pd.read_csv('ProcessedReviews.csv')

In [ ]:
def binaryReview(rating):
    if rating > 3:
        return 1
    if rating < 3:
        return 0

skiAreaReviewDF['Binary Rating'] = skiAreaReviewDF['Review Star Rating (out of 5)'].apply(lambda rating: binaryReview(rating))
skiAreaReviewDF = skiAreaReviewDF[skiAreaReviewDF['Binary Rating'].notnull()]

## Stemmed

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer() 
X = cv.fit_transform(skiAreaReviewDF['Clean Reviews Stemmed'].to_list())

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, skiAreaReviewDF['Binary Rating'], test_size=0.33, random_state=42, stratify=skiAreaReviewDF['Binary Rating'].to_list())

In [10]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000, criterion='entropy', random_state=42)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5)
scores

In [ ]:
sum(scores)/float(len(scores))

## Lemmed

In [ ]:
cv = CountVectorizer() 
X = cv.fit_transform(skiAreaReviewDF['Clean Reviews Lemmed'].to_list())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, skiAreaReviewDF['Binary Rating'], test_size=0.33, random_state=42, stratify=skiAreaReviewDF['Binary Rating'].to_list())

In [ ]:
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5)
scores

In [ ]:
sum(scores)/float(len(scores))

## Lemmed and Stemmed

In [45]:
cv = CountVectorizer() 
X = cv.fit_transform(skiAreaReviewDF['Clean Reviews Lemmed and Stemmed'].to_list())

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    X, skiAreaReviewDF['Binary Rating'], test_size=0.33, random_state=42, stratify=skiAreaReviewDF['Binary Rating'].to_list())

In [48]:
result = model.fit(X_train, y_train)

In [52]:
result.score(X_train, y_train)

0.9945365311602926

In [47]:
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5)
scores

array([0.83518519, 0.83564815, 0.83240741, 0.83518519, 0.83371931])

In [ ]:
sum(scores)/float(len(scores))

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()
X = tf.fit_transform(skiAreaReviewDF['Clean Reviews Lemmed and Stemmed'].to_list())

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, skiAreaReviewDF['Binary Rating'], test_size=0.33, random_state=42, stratify=skiAreaReviewDF['Binary Rating'].to_list())

In [11]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5)
scores

array([0.83564815, 0.83472222, 0.83333333, 0.83611111, 0.83325614])

In [12]:
sum(scores)/float(len(scores))

0.8346141903830648

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
tft = TfidfTransformer()
X = tft.fit_transform(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(
    X, skiAreaReviewDF['Binary Rating'], test_size=0.33, random_state=42, stratify=skiAreaReviewDF['Binary Rating'].to_list())

In [16]:
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=5)
scores

array([0.83564815, 0.83518519, 0.83333333, 0.83611111, 0.83325614])

In [17]:
sum(scores)/float(len(scores))

0.8347067829756576

In [18]:
from sklearn.model_selection import GridSearchCV
parameters =[{'n_estimators': [10, 100, 1000], 'criterion':['entropy', 'gini'], 'max_features':[2,3,4,5]}]

In [19]:
gridSearch = GridSearchCV(model, parameters, cv=5, scoring='accuracy', return_train_score=True)

In [ ]:
gridSearch.fit(X_train, y_train)

In [ ]:
gridSearch.best_params_

In [ ]:
cvResults = gridSearch.cv_results_

In [ ]:
for accuracy, parameters in zip(cvResults['mean_test_score'], cvResults['params']):
    print(accuracy, parameters)